# Introduzione a Tensorflow

TensorFlow (TF) è la API di Google per lo sviluppo di modelli di Deep Learning. Interagiremo con TF in Python, ma è possibile usare altri linguaggi, almeno a basso livello. La tipica applicazione TF lavora secondo lo schema riportato di seguito:

<img src="https://pic1.zhimg.com/80/v2-8b46a7f55b77f0febfa3ad5084e25c3c_1440w.jpg" alt="Architettura TF" width="50%" />

- Keras è una nota libreria per specificare ad alto livello i layer del modello che lavora anche con altri back-end
- La Data API è il modulo `tf.data` di TF che serve a specificare le azioni da compiere sul data set ai fini di addestramento e test
- L'Execution Engine di TF (che può essere locale o anche distribuito su più nodi di un cluster di calcolo) rappresenta il nucleo vero e proprio di TF. A questo livello, vengono esposte API in diversi linguaggi.

Nell'execution engine, viene definita la _sessione_ di lavoro `tf.Session(...)` in cui si svolgono le operazioni atomiche (`tf.Operation`) sui dati in forma tensoriale (`tf.Tensor`). 

La sessione gestisce l'esecuzione di un _*grafo di computaizone*_ ovvero di una specifica _astratta_ della sequenza di operazioni secondo una struttura a grafo dove i nodi sono operazioni che restituiscono tensori che vanno in ingresso ad altri nodi secondo la struttura definita dagli archi.

<img src="https://miro.medium.com/max/2994/1*vPb9E0Yd1QUAD0oFmAgaOw.png" alt="Esempio grafo computazione" width="50%" />

L'esecuzione si ottiene attraverso il metodo `tf.Operation.run()` che è ereditato anche dalla sessione ovvero si può valutare un singolo tensore con `tf.Tensor.eval()`. La sessione va chiusa esplicitamente con il metodo `close()`.



In [1]:
import tensorflow as tf
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


# Sostituisce la vecchia chiamata tf.Session() 
# che è deprecata da quando è stato rilasciato TF v.2
sess = tf.compat.v1.Session()

# chiediamo la lista dei device di calcolo presenti
# quelli di tipo XLA_GPU o XLA_CPU sono abilitati all'algebra lineare accelerata
for d in sess.list_devices():
    print(d.name)

# Definiamo una semplice computazione che definisce un elemento (o meglio un **nodo**)
# costante di tipo stringa e lo esegue ottenendone la stampa
bye_bye = tf.constant('Hello World')
result = sess.run(bye_bye)

print(result)

print(f'The session is closed? {sess._closed}')
# chiudiamo la sessione
sess.close()

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:XLA_CPU:0
/job:localhost/replica:0/task:0/device:XLA_GPU:0
/job:localhost/replica:0/task:0/device:GPU:0
b'Hello World'
The session is closed? False


La sessione può essere invocata con diverse opzioni e, soprattutto, con la possibilità di definire un _context manager_ all'interno del quale specificare le operazioni che non richiede più la chiusura esplicita.

In [2]:

with tf.compat.v1.Session() as sess:
    # definizione di due valori costanti interi
    n1=tf.constant(2)
    n2=tf.constant(3)
    
    print(n1.eval())
    
     #n3 = n1 * n2    # questa **non è** la moltiplicaione tra interi, 
                    # ma la tf.Operation di moltiplicazione tra tensori
    
    n3 = tf.multiply(n1,n2)
        
    print(n1,n2,n3,sep='\n')
    
    #print(sess.run(n3))
    print(n3.eval(), n3.dtype, n3.shape)

print(f'Session closed: {sess._closed}')

2
Tensor("Const_1:0", shape=(), dtype=int32)
Tensor("Const_2:0", shape=(), dtype=int32)
Tensor("Mul:0", shape=(), dtype=int32)
6 <dtype: 'int32'> ()
Session closed: True


La sintassi completa del costruttore dell'oggetto `tf.compat.v1.Session` è:

```python
tf.compat.v1.Session(target='',\        # engine di computazione da utilizzare, locale o distribuito
                    graph=None,\        # grafo di computazione da utilizzare
                    config=None)        # oggetto con le specifiche particolari di configurazione
```

Apriamo la sessione con una configurazione di default per l'allocazopne dinamica della computazione sui diversi device disponibili ed eseguiamo il grafo di computazione della figura precedente

In [1]:
import tensorflow as tf
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Usiamo esplicitamente l'oggetto di configurazione

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(
    allow_soft_placement=True,      # gestione dinamica dell'allocazione dei device
    log_device_placement=True))     # registrazione del log sull'allocazione dei device

with sess.as_default():
    assert tf.compat.v1.get_default_session() is sess # impostiamo la nostra sessione configurata come quella di default
    
    # La nostra computazione è: res = (a*b) / (a+b)
    
    # ingressi
    a = tf.constant(5)
    b = tf.constant(3)
    
    # operazioni intermedie
    prod = tf.multiply(a,b)
    sum = tf.add(a,b)
    
    # uscita
    #res = tf.div(prod,sum)
    res = tf.math.divide(prod,sum)

    
    #print(res.eval())
    print(sess.run(res))


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN Xp, pci bus id: 0000:03:00.0, compute capability: 6.1

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN Xp, pci bus id: 0000:03:00.0, compute capability: 6.1
Mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
Add: (Add): /job:localhost/replica:0/task:0/device:GPU:0
truediv/Cast: (Cast): /job:localhost/replica:0/task:0/device:GPU:0
truediv/Cast_1: (Cast): /job:localhost/replica:0/task:0/device:GPU:0
truediv: (RealDiv): /job:localhost/replica:0/task:0/device:GPU:0
Const: (Const): /job:localhost/replica:0/task:0/device:GPU:0
Const_1: (Const): /job:localhost/replica:

In [2]:
# Creiamo tensori con diverse funzioni
mat = tf.constant([[1., 2., 3.], [4., 5., 6.]])

print(mat.shape, mat.dtype)

(2, 3) <dtype: 'float32'>


In [3]:
mat_randn = tf.random.normal((3,3), mean=0, stddev=1.0)	            # A 3ｘ3 random normal matrix.
mat_randu = tf.random.uniform((4,4), minval=0, maxval=1.0)

print(mat_randn)

with sess.as_default():
    assert tf.compat.v1.get_default_session() is sess
    
    
    print(mat_randn.eval())
    print(mat_randu.eval())
    

Tensor("random_normal:0", shape=(3, 3), dtype=float32)
Mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
Add: (Add): /job:localhost/replica:0/task:0/device:GPU:0
truediv/Cast: (Cast): /job:localhost/replica:0/task:0/device:GPU:0
truediv/Cast_1: (Cast): /job:localhost/replica:0/task:0/device:GPU:0
truediv: (RealDiv): /job:localhost/replica:0/task:0/device:GPU:0
random_normal/RandomStandardNormal: (RandomStandardNormal): /job:localhost/replica:0/task:0/device:GPU:0
random_normal/mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
random_normal: (Add): /job:localhost/replica:0/task:0/device:GPU:0
random_uniform/RandomUniform: (RandomUniform): /job:localhost/replica:0/task:0/device:GPU:0
random_uniform/sub: (Sub): /job:localhost/replica:0/task:0/device:GPU:0
random_uniform/mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
random_uniform: (Add): /job:localhost/replica:0/task:0/device:GPU:0
Const: (Const): /job:localhost/replica:0/task:0/device:GPU:0
Const_1: (Const): /job

In [6]:
# Esempio di uso delle variabili
with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True)) as sess:

    init_values = tf.random.uniform((4,4), minval=0, maxval=1.0)

    t = tf.Variable(initial_value=init_values,name='myvar')

    init = tf.compat.v1.global_variables_initializer()
    
    print(sess.run(init))
    print(sess.run(t))


None
[[0.5368296  0.5577097  0.95285404 0.72499657]
 [0.92383134 0.3147235  0.07189012 0.5213605 ]
 [0.9221605  0.7092     0.36317253 0.6396687 ]
 [0.23651493 0.44773293 0.67429364 0.56049836]]


In [7]:
# Definiamo i placeholder per z = 2x^2 + 2xy
with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True)) as sess:

    two = tf.constant(2.0)
    
    x = tf.compat.v1.placeholder(tf.float32,shape=(None, 3))
    
    y = tf.compat.v1.placeholder(tf.float32,shape=(None, 3))
    
    z = tf.add(tf.multiply(two, tf.multiply(x, x)),\
                tf.multiply(two, tf.multiply(x, y)))
    
    print(sess.run(z, feed_dict={x: [[1., 2., 3.],[4., 5., 6.]], y: [[3., 4., 5.],[7., 8., 9.]]}))


[[  8.  24.  48.]
 [ 88. 130. 180.]]
